# Магические методы классов

## Строковое представление объектов

- `__repr__` - строковое представление объектов
- `__str__` - метод, который вызывается функциями `str`, `format`, `print`
- `__format__` - метод, который вызывается при форматировании строки (`.format(...)`, f-strings)

In [1]:
class A:
    def __init__(self, value):
        self.value = value
    
    def __str__(self):
        return f'A({self.value}) object str method'
    
    def __repr__(self):
        return f'A({self.value}) object repr method'
    
    def __format__(self, format_spec):
        print(format_spec)
        return f'A({self.value})'
    

a = A(365)
str(a)
print(a)
print("some text with %s" % a)
print(f"some text with {a}")
print("some text with {a}".format(a=a))
a

A(365) object str method
some text with A(365) object str method

some text with A(365)

some text with A(365)


A(365) object repr method

## Rich Comparison

Магические методы для использования объекта класса с операторами сравнения. Нужно либо указать все, либо использовать декоратор total_ordering, который дополнит недостающие методы "антонимами":

In [4]:
from functools import total_ordering

@total_ordering
class Person(object):

    def __init__(self, firstname, lastname):
        self.first = firstname
        self.last = lastname

    def __eq__(self, other):
        print(f"{self.first} {self.last} eq:", end='\t')
        return ((self.last, self.first) == (other.last, other.first))

    def __ne__(self, other):
        print(f"{self.first} {self.last} ne:", end='\t')
        return ((self.last, self.first) != (other.last, other.first))

    def __lt__(self, other):
        print(f"{self.first} {self.last} lt:", end='\t')
        return ((self.last, self.first) < (other.last, other.first))

#     def __le__(self, other):
#         print(f"{self.first} {self.last} le:", end='\t')
#         return ((self.last, self.first) <= (other.last, other.first))

#     def __gt__(self, other):
#         print(f"{self.first} {self.last} gt:", end='\t')
#         return ((self.last, self.first) > (other.last, other.first))

#     def __ge__(self, other):
#         print(f"{self.first} {self.last} ge:", end='\t')
#         return ((self.last, self.first) >= (other.last, other.first))

    def __repr__(self):
        return "%s %s" % (self.first, self.last)
    
    
a = Person("Евпатий", "Коловрат")
b = Person("Илья", "Муромец")

print(a == b)
print(a != b)
print(a >= b)
print(a > b)
print(a <= b)
print(a < b)

Евпатий Коловрат eq:	False
Евпатий Коловрат ne:	True
Евпатий Коловрат lt:	False
Евпатий Коловрат lt:	False
Евпатий Коловрат lt:	True
Евпатий Коловрат lt:	True


### Метод `__hash__`

Этот метод отвечает за уникальность объекта и возможность его использования как ключ в hashable-коллекциях. Создадим пустой класс и посмотрим, как в нем работают оператор `is`, использование его как ключа в hashable-коллекции и сравнение.

In [5]:
class X:
    pass

a = X()
b = X()

print(a is b)
print({a: 1, b: 2})
print(a == b)
print(hash(a), hash(b))

False
{<__main__.X object at 0x000001AB09D3BF98>: 1, <__main__.X object at 0x000001AB09D3BCF8>: 2}
False
-9223371922222531591 -9223371922222531633


Добавим метод, который делает наш объект равным чему угодно.

In [10]:
class X:
    def __eq__(self, other):
        return True
    
#     def __hash__(self):
#         return 25
    
    def __str__(self):
        return str(id(self))
    
    __repr__ = __str__


a = X()
b = X()

print(a == b)
print(a is b)
print({a: 1, b: 2})
print(hash(a), hash(b))

True
False


TypeError: unhashable type: 'X'

Взять хеш от объектов не получилось, поскольку при определенном методе `__eq__` сравнение объектов по уникальности неочевидно для интерпретатора. Необходимо реализовать собственный метод `__hash__` в классе. Основное свойство хеша состоит в том, что он должен быть разным для разных объектов. Но что будет, если не придерживаться этого свойства? 

In [11]:
X.__hash__ = lambda self: 25

print(a == b)
print(a is b)
print({a: 1, b: 2})
print(hash(a), hash(b))

True
False
{1834115968584: 2}
25 25


In [12]:
id(a)

1834115968584

Значение в словаре перезаписалось, поскольку хеш от обоих объектов одинаковый. Оператор `is` тем не менее выдает False, поскольку id объектов разные.

## Работа с числовыми операторами

https://habr.com/ru/post/186608/

### Преобразование типов
- `__int__(self)` - Преобразование типа в int.
- `__long__(self)` - Преобразование типа в long.
- `__float__(self)` - Преобразование типа в float.
- `__complex__(self)` - Преобразование типа в комплексное число.
- `__oct__(self)` - Преобразование типа в восьмеричное число.
- `__hex__(self)` - Преобразование типа в шестнадцатиричное число.
- `__index__(self)` - Преобразование типа к int, когда объект используется в срезах (выражения вида `[start:stop:step]`). Если вы определяете свой числовый тип, который может использоваться как индекс списка, вы должны определить `__index__`.
- `__trunc__(self)` - Вызывается при math.trunc(self). Должен вернуть своё значение, обрезанное до целочисленного типа (обычно long).


### Унарные операции

- `__pos__(self)` - Определяет поведение для унарного плюса (+some_object)
- `__neg__(self)` - Определяет поведение для отрицания(-some_object)
- `__abs__(self)` - Определяет поведение для встроенной функции abs().- `__invert__(self)` - Определяет поведение для инвертирования оператором ~. Для объяснения что он делает смотри статью в Википедии о бинарных операторах.
- `__round__(self, n)` - Определяет поведение для встроенной функции round(). n это число знаков после запятой, до которого округлить.
- `__floor__(self)` - Определяет поведение для math.floor(), то есть, округления до ближайшего меньшего целого.
- `__ceil__(self)` - Определяет поведение для math.ceil(), то есть, округления до ближайшего большего целого.- `__trunc__(self)` - Определяет поведение для math.trunc(), то есть, обрезания до целого.


### Бинарные операции


Следующие операции вызываются над левым операндом в операторах. Т.е., например, `x + y` - это `x.__add__(y)`.

- `__add__(self, other)` - Сложение.
- `__sub__(self, other)` - Вычитание.
- `__mul__(self, other)` - Умножение.
- `__matmul__(self, other)` - Оператор перемножения матриц `@`
- `__floordiv__(self, other)` - Целочисленное деление, оператор `//`.
- `__truediv__(self, other)` - Деление, оператор `/`.
- `__mod__(self, other)` - Остаток от деления, оператор %.
- `__divmod__(self, other)` - Определяет поведение для встроенной функции divmod().
- `__pow__` - Возведение в степень, оператор `**`.
- `__lshift__(self, other)` - Двоичный сдвиг влево, оператор `<<`.
- `__rshift__(self, other)` - Двоичный сдвиг вправо, оператор `>>`.
- `__and__(self, other)` - Двоичное И, оператор `&`.
- `__or__(self, other)` - Двоичное ИЛИ, оператор `|`.
- `__xor__(self, other)` - Двоичный xor, оператор `^`.

Кроме того, есть функции, с префиксом `r`, например, `__radd__`. Это аналоги тех же функций, но только для тех случаев, когда операнд стоит справа. Т.е. `x + y` - это `y.__radd__(x)`.

И также есть префикс `i`, например, `__iadd__`, для случаев операций с присвоением (`x += y`).

In [ ]:
class Meal:
    def __init__(self, title, price):
        self.title = title
        self.price = price
        
    def __str__(self):
        return ': '.join([self.title, str(self.price)])
    
    def __repr__(self):
        """Функция, которая используется для текстового представления объекта в случаях, когда это происходит не
        через функцию str(obj)"""
        return str(self)
    
    def __add__(self, other):
        """Функция, которая описывает прибавление к нашему объекту объекта other"""
        # если у нас оба объекта данного класса, сложим их атрибуты
        if isinstance(other, Meal):
            new_title = ', '.join([self.title, other.title])
            new_price = self.price + other.price
        else:
            # а если второй объект не этого класса, то попробуем его привести к типу float
            new_title = self.title + " и что-то еще"
            new_price = self.price + float(other)
        self.title = new_title
        self.price = new_price
        return self
    
    def __iadd__(self, other):
        print(f"Нельзя впихнуть в {self.title} другую еду")
        return self
    
    def __neg__(self):
        self.price = -self.price
        return self

In [ ]:
Meal("БигМак", 200)

In [ ]:
Meal("БигМак", 200) + Meal("Картошка", 50)

In [ ]:
Meal("БигМак", 200) + 25

In [ ]:
25 + Meal("БигМак", 200)

Произошла ошибка, поскольку у нас определено сложение только <code>Meal + что-то</code>, но не <code>что-то + Meal</code>. В случаях, когда складываемые объекты разных типов, операция сложения в питоне некоммутативна. Чтобы определить обратное сложение, добавим метод <code>\__radd__</code>

In [ ]:
Meal.__radd__ = Meal.__add__

25 + Meal("БигМак", 200)

In [ ]:
- Meal("Еда", 2000)

In [ ]:
a = Meal('доширак', 40)
a += 1000
a

## Эмуляция контейнеров

- `object.__len__(self)` - длина контейнера
- `object.__getitem__(self, key)` - получение элемента по индексу/ключу
- `object.__setitem__(self, key, value)` - назначение элемента
- `object.__delitem__(self, key)` - удаление элемента
- `object.__missing__(self, key)` - этот метод определен только для классов-наследников словаря и вызывается методом `__getattr__`, когда в словаре нет требуемого элемента
- `object.__iter__(self)` - получение итератора
- `object.__reversed__(self)` - обратный порядок элементов
- `object.__contains__(self, item)` - оператор in

In [ ]:
class NumberDict(dict):
    def __getitem__(self, key):
        return super(NumberDict, self).__getitem__(float(key))
    
    def __setitem__(self, key, value):
        super(NumberDict, self).__setitem__(float(key), value)
        
    def __delitem__(self, key):
        super(NumberDict, self).__delitem__(float(key))
         
    def __missing__(self, key):
        return 'no-no-no'
    
    def __contains__(self, item):
        return self[item] != self.__missing__(1)


my_dict = NumberDict()  
print(my_dict["0"])

my_dict["0"] = 1
print(my_dict[0])

print(1 in my_dict)
print("0.000" in my_dict)

## Контроль объектов

- `__dir__(self)` - Определяет поведение функции `dir()`, вызванной на экземпляре вашего класса. Этот метод должен возвращать пользователю список атрибутов. Обычно, определение `__dir__` не требуется, но может быть жизненно важно для интерактивного использования вашего класса, если вы переопределили `__getattr__` или `__getattribute__` (с которыми вы встретитесь в следующей части), или каким-либо другим образом динамически создаёте атрибуты.
- `__sizeof__(self)` - Определяет поведение функции `sys.getsizeof()`, вызыванной на экземпляре вашего класса. Метод должен вернуть размер вашего объекта в байтах. Он главным образом полезен для классов, определённых в расширениях на C, но всё-равно полезно о нём знать.

Дурацкий пример `__dir__`:

In [13]:
class SlotsClass:
    __slots__ = ('foo', 'bar')
    
    def __dir__(self):
        return list(self.__slots__)
    
dir(SlotsClass())

['bar', 'foo']

In [14]:
a = [1, 2, 3, 4]

In [ ]:
a.append